In [13]:
using JLD
include("../data/synthetic.jl")
using .Synthetic
using TOML
using Gen
using PyPlot
using LinearAlgebra
using ProgressBars
using Statistics
using Distributions
using CSV
using Random
using DataFrames

In [14]:
include("../src/model.jl")
include("../src/estimation.jl")
include("../src/inference.jl")
include("../data/processing_IHDP.jl")

using .Model
using .Estimation
using .Inference
using .ProcessingIHDP

In [67]:
logmeanexp(x) = logsumexp(x)-log(length(x))

# +
experiment = 1
experiments = [i for i in experiment*10-9:experiment*10]

config_paths = ["../experiments/config/IHDP/$(experiment).toml" for experiment in experiments]
configs = [TOML.parsefile(config_path) for config_path in config_paths]
config = configs[1]
Random.seed!(config["data_params"]["seed"])

data = DataFrame(CSV.File(config["paths"]["data"]))[1:config["data_params"]["nData"], :]
pairs = ProcessingIHDP.generatePairs(data, config["data_params"]["pPair"])
nData = size(data)[1]
n = nData + length(pairs)

SigmaU = ProcessingIHDP.generateSigmaU(pairs, nData)
T_ = ProcessingIHDP.generateT(data, pairs)
T = [Bool(t) for t in T_]
doT = [Bool(1-t) for t in T_]
X_ = ProcessingIHDP.generateX(data, pairs)
nX = size(X_)[2]
X = [X_[!, i] for i in 1:nX]
U = ProcessingIHDP.generateU(data, pairs)
BetaX, BetaU = ProcessingIHDP.generateWeights(config["data_params"]["weights"], config["data_params"]["weightsP"])
Y_, Ycf = ProcessingIHDP.generateOutcomes(X_, U, T_, BetaX, BetaU, config["data_params"]["CATT"], n)
Y = [Float64(y) for y in Y_]
print()

In [75]:
obj_label = vcat([i for i in 1:200], pairs)

# generate initial parameters from prior
# ty, xy, uy, ynoise
@gen function generateLS(mean, scale)
    LS = @trace(normal(mean, scale), :LS)
    return LS
end
@gen function generateNoise(shape, scale)
    Noise = @trace(inv_gamma(shape, scale), :Noise)
    return Noise
end
nObj = 200
MappedGenerateLS = Map(generateLS)
MappedGenerateNoise = Map(generateNoise)

@gen function thetaProposal(trace, var::Float64)
    mu = trace[:theta]
    @trace(normal(mu, var), :theta)
end

@gen function alphaProposal(trace, i::Int, var::Float64)
    mu = trace[:alpha => i => :LS]
    @trace(normal(mu, var), :alpha => i => :LS)
end

@gen function betaProposal(trace, i::Int, var::Float64)
    mu = trace[:beta => i => :LS]
    @trace(normal(mu, var), :beta => i => :LS)
end

@gen function NoiseProposal(trace, var::Float64)
    cur = trace[:noise]
    
    Shape = (cur * cur / var) + 2
    Scale = cur * (Shape - 1)
    
    @trace(inv_gamma(Shape, Scale), :noise)
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any, Float64], ##NoiseProposal#9174, Bool[0, 0], false)

In [87]:
@gen function LinearMLMOffset(xs::Vector{Float64}, ts::Vector{Float64}, obj_label, nObj)
    n, nX = size(xs)
    beta =  @trace(MappedGenerateLS(fill(0.0, nX), fill(1.0, nX)), :beta) #xyLS
    theta = @trace(normal(0,1), :theta) #tyLS
    alpha = @trace(MappedGenerateLS(fill(0.0, nObj), fill(10.0, nObj)), :alpha) #alpha
    sigma = @trace(inv_gamma(4.0, 4.0), :noise) #tyLS
    for i in 1:n
        obj = Int(obj_label[i])
        t = ts[i]
        x = Array(xs[i, :])
        @trace(normal( sum(beta.*x) +theta * t + alpha[obj], sigma), "y-$i")
    end
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Array{Float64,1}, Array{Float64,1}, Any, Any], ##LinearMLMOffset#9176, Bool[0, 0, 0, 0], false)

In [89]:
constraints = choicemap()
for (i, y) in enumerate(Y)
    constraints["y-$i"] = y
end
n_run = 1000
PosteriorSamples = []
(trace, _) = generate(LinearMLMOffset, (X_, T, obj_label, nObj), constraints)
for iter=tqdm(1:n_run)
    (trace, _) = mh(trace, thetaProposal, (0.5, ))
    (trace, _) = mh(trace, NoiseProposal, (0.5, ))
    for k in 1:nObj
        (trace, _) = mh(trace, alphaProposal, (k, 0.5))
    end 
    for k in 1:nX
        (trace, _) = mh(trace, betaProposal,  (k, 0.5))
    end 
    
    push!(PosteriorSamples, get_choices(trace))
end

100.00%┣█████████████████████████████████████████████████████████▉┫ 1000/1000 14:05<00:00, 1.18 it/s]


In [262]:
# evaluarion
Ycfs, mask, doT = [], nothing, nothing
preds = []
avg, noises = nothing, [] 
avg_noises= []
doTs = [1.0, 0.0]
for (i, doT) in tqdm(enumerate(doTs))
    mask = T .!= doT
    push!(Ycfs, Ycf[mask])
    pred_at_doT = []
    noises = []
    for j in 801:n_run
        theta = PosteriorSamples[j][:theta]
        beta = [PosteriorSamples[j][:beta=>k=>:LS] for k in 1:nX]
        alpha = [PosteriorSamples[j][:alpha=>k=>:LS] for k in 1:nObj]
        noise = PosteriorSamples[j][:noise]
        Ypred = (Array(X_[mask, :]) * beta .+ theta * doT .+ alpha[Int.(obj_label[mask])])
   
        push!(pred_at_doT, Ypred)
        push!(noises, noise)
    end
#     avg = [mean([pred_at_doT[i][j] for i in 1:200]) for j in 1:length(pred_at_doT[i])]
    push!(preds, pred_at_doT)
end

100.00%┣████████████████████████████████████████████████████████████████┫ 2/2 00:00<00:00, 4.52 it/s]


In [359]:
for k in 1:length(doTs)
    mask = T .!= doTs[k]
    Ycf_pred = []
    for i in 1:sum(mask)
        preddoT = []
        for j in 1:200
            push!(preddoT, preds[k][j][i])
        end
        push!(Ycf_pred, mean(preddoT))
    end   
    println(doTs[k], " ", mean((Ycfs[k] .- Ycf_pred) .^ 2)^0.5)
end 

1.0 4.7602651042621
0.0 3.2373659368346654


In [361]:
truthLogLikelihoods = Dict()
mu = nothing
for k in 1:length(doTs)
    doT = doTs[k]
    truthLogLikelihoods[doT] = []
    mask = T .!= doT
    for i in 1:200
        mu = [preds[k][i][j] for j in 1:sum(mask)] 
        sigma = noises[i] * Matrix{Float64}(I, sum(mask), sum(mask))
        truthLogLikelihood = Distributions.logpdf(MvNormal(mu, sigma), Ycf[mask])
        push!(truthLogLikelihoods[doT], truthLogLikelihood)
    end 
    println(doT, " ", logmeanexp([Real(llh/sum(mask)) for llh in truthLogLikelihoods[doT]]))
end

1.0 -5.226586046576538
0.0 -3.4847045522816193
